In [1]:
%set_env ANTHROPIC_API_KEY=${ANTHROPIC_API_KEY}
%set_env LANGSMITH_TRACING=true
%set_env LANGSMITH_API_KEY=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
%set_env TAVILY_API_KEY=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


%set_env OPENAI_API_KEY=${OPENAI_API_KEY}
%set_env LANGCHAIN_API_KEY=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

%set_env LANGCHAIN_TRACING_V2=true
%set_env LANGCHAIN_PROJECT="Multi-agent Collaboration"

env: ANTHROPIC_API_KEY=${ANTHROPIC_API_KEY}
env: LANGSMITH_TRACING=true
env: LANGSMITH_API_KEY=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
env: TAVILY_API_KEY=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
env: OPENAI_API_KEY=${OPENAI_API_KEY}
env: LANGCHAIN_API_KEY=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
env: LANGCHAIN_TRACING_V2=true
env: LANGCHAIN_PROJECT="Multi-agent Collaboration"


In [2]:
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import pandas as pd
import pandasql as psql


def ChatAnthropicSKT(model="anthropic/claude-3-5-sonnet-20240620"):
    llm_api_key = "sk-gapk-1tQCB4O2KnH5GG68DeUKfjAKQ-vJ9kc9"
    llm_api_url = "https://api.platform.a15t.com/v1"

    # model = "anthropic/claude-3-5-sonnet-20240620"

    model = ChatOpenAI(
        temperature=0,  
        openai_api_key=llm_api_key, 
        openai_api_base=llm_api_url, 
        model=model
        )
    return model

llm = ChatAnthropicSKT()

# llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

# llm = ChatOpenAI(model="gpt-3.5-turbo")
# llm = ChatAnthropicSKT(model="azure/openai/gpt-4-1106")


In [3]:
import json
from typing import Annotated, List

from langchain_core.tools import tool

from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Dict, Optional

from typing_extensions import TypedDict

@tool
def check_schema(
    # dataframe_json: Annotated[str, "dataframe in JSON format."], 
    dataframe_path: Annotated[str, "path to the dataframe."]
):    
    """
    Check the schema of a dataframe.
    
    Args:
        # dataframe_json (str): The dataframe in JSON format.
        dataframe_path (str): The path to the dataframe.
    
    Returns:
        str: The schema of the dataframe.
    """

    # print("=====check_schema>dataframe_json: ",dataframe_json)
    # print("=====check_schema>dataframe_path: ",dataframe_path)

    try:
        # If it's a string, try to parse it as JSON
        # try:
        #     if isinstance(dataframe_json, str):
        #         json_data = json.loads(dataframe_json)
        #     else:
        #         json_data = dataframe_json
            
        #     # Create a DataFrame
        #     df = pd.DataFrame(json_data)
        # except:
        #     df = pd.read_csv(dataframe_path)

        df = pd.read_csv(dataframe_path)
        
        # Get the schema
        schema = df.dtypes.to_string()
        return f"DataFrame schema:\n{schema}"
    except json.JSONDecodeError as e:
        error_msg = f"Error decoding JSON string: {str(e)}"
        return error_msg
    except Exception as e:
        error_msg = f"Error checking schema: {str(e)}"
        return error_msg

@tool
def generate_sql_query(
    user_request: Annotated[str,"user request to generate a sql query"],
    dataframe_schema: Annotated[str,"dataframe schema to generate a sql query"],
    dataframe_name: Annotated[str,"dataframe name to search"]
) -> str:
    """
    A tool to generate a SQL query based on the user request and schema in the message.
    """

    # print("=====generate_sql_query>dataframe_schema: ",dataframe_schema)
    # print("=====generate_sql_query>dataframe_name: ",dataframe_name)

    try:
        prompt = f"Given the following user request:'{user_request}' and schema:\n{dataframe_schema} and table name:'{dataframe_name}'\n\nGenerate a SQL query for the following message."
        sql_query = llm.invoke([HumanMessage(content=prompt)]).content
        return sql_query
    except Exception as e:
        return ""

# @tool
# def search_dataframe(
#     dataframe_json: Annotated[str, "dataframe in JSON format."], 
#     sql_query: Annotated[str, "SQL query to execute."]
# ) -> str:
#     """
#     A tool to search a dataframe using a SQL query from SQL_Expert.
#     """
#     print("=====search_dataframe>dataframe_json: ",dataframe_json)
#     print("=====search_dataframe>sql_query: ",sql_query)
#     try:
#         df = pd.read_json(dataframe_json)
#         result = psql.sqldf(sql_query, locals())
#         return result.to_json(orient='records')
#     except Exception as e:
#         return "[]"

@tool
def search_dataframe(
    dataframe_path: Annotated[str, "path to the dataframe."],
    sql_query: Annotated[str, "SQL query to execute."]
) -> str:
    """
    A tool to search a dataframe using a SQL query from SQL_Expert.
    """
    # print("=====search_dataframe>dataframe_path: ",dataframe_path)
    # print("=====search_dataframe>sql_query: ",sql_query)
    try:
        df = pd.read_csv(dataframe_path)
        # print("=====search_dataframe>result: ",stock_data_krx)
        result = psql.sqldf(sql_query, locals())
        return result.to_json(orient='records')
    except Exception as e:
        return "[]"

In [4]:
from typing import Annotated

from langchain_anthropic import ChatAnthropic
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


# Modification: tell the LLM which tools it can call
tools = [check_schema, generate_sql_query, search_dataframe]
llm_with_tools = llm.bind_tools(tools)


def supervisor_agent(state: State):
    # print("supervisor_agent state[messages]: ", state["messages"])
    # print("supervisor_agent invoke result: ", llm_with_tools.invoke(state["messages"]))
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("supervisor_agent", supervisor_agent)

In [5]:
import json

from langchain_core.messages import ToolMessage


class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    # The inputs are the state of the graph at the time of the node being called
    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        # print("tool node input: ", inputs)
        outputs = []
        for tool_call in message.tool_calls: # tool_call : {'name': 'tavily_search_results_json', 'args': {'query': 'langchain'}, 'id': 'toolu_0196pDFQjAVMseyd5RX2vnqv', 'type': 'tool_call'}
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        # print("tool node outputs: ", outputs)
        return {"messages": outputs}


tool_node = BasicToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

In [6]:
from typing import Literal


def route_tools(
    state: State,
) -> Literal["tools", "__end__"]:
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return "__end__"


# The `tools_condition` function returns "tools" if the supervisor_agent asks to use a tool, and "__end__" if
# it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "supervisor_agent",
    route_tools,
    # The following dictionary lets you tell the graph to interpret the condition's outputs as a specific node
    # It defaults to the identity function, but if you
    # want to use a node named something else apart from "tools",
    # You can update the value of the dictionary to something else
    # e.g., "tools": "my_tools"
    {"tools": "tools", "__end__": "__end__"},
)
# Any time a tool is called, we return to the supervisor_agent to decide the next step
graph_builder.add_edge("tools", "supervisor_agent")
graph_builder.add_edge(START, "supervisor_agent")
graph = graph_builder.compile()

In [7]:
'''
Search the latest close price of symbol '000100.KS'.
Use the following information:
    dataframe_path="data/stock_data_krx.csv"
    dataframe_name='df'
'''

from langchain_core.messages import BaseMessage

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            if isinstance(value["messages"][-1], BaseMessage):
                # print("Assistant:", value["messages"][-1].content)
                value["messages"][-1].pretty_print()

================================== Ai Message ==================================

Certainly! I'll search for the latest close price of the symbol '000100.KS' using the information you provided. To do this, we'll need to use a couple of tools in sequence. First, we'll check the schema of the dataframe, then generate a SQL query, and finally search the dataframe with that query.

Let's start by checking the schema of the dataframe:
Tool Calls:
  check_schema (toolu_01RZNCq5rqvuQfvPGvgSG62f)
 Call ID: toolu_01RZNCq5rqvuQfvPGvgSG62f
  Args:
    dataframe_path: data/stock_data_krx.csv
tool node input:  {'messages': [HumanMessage(content='Search the latest close price of symbol \'000100.KS\'. Use the following information:     dataframe_path="data/stock_data_krx.csv"     dataframe_name=\'df\'', id='1e709340-cbaf-48cf-9764-76bd7996a0c0'), AIMessage(content="Certainly! I'll search for the latest close price of the symbol '000100.KS' using the information you provided. To do this, we'll need to